Custom load

In [1]:
import tensorflow as tf
import os, json, h5py
import numpy as np

In [2]:
def build_model(input_shape, num_classes):
    
    in_layer = tf.keras.layers.Input(input_shape, name="in_layer")
    
    x = in_layer
    x = tf.keras.layers.Conv2D(16, (3, 3), padding="valid", name="l1_conv2d")(x)
    x = tf.keras.layers.Conv2D(32, (3, 3), padding="valid", name="l2_conv2d")(x)
    x = tf.keras.layers.Flatten(name="l3_flatten")(x)
    
    out_layer = tf.keras.layers.Dense(num_classes, activation="softmax", name="out_layer")(x)
    
    return tf.keras.models.Model(inputs=in_layer, outputs=out_layer)

In [21]:
# def load_weights(model, weights):
#     with h5py.File("./model1.h5", "r") as weights:
#         for l in model.layers:
#             for w in l.trainable_weights:
#                 tf.keras.backend.set_value(w, weights["{}/{}".format(l.name, w.name)])

def load_weights(model, weights_file):
    """
        Load weights from saved file based on layer name.
        This is temporay solution for the problem discussed in
        https://spectrum.chat/larq/general/testing-binary-models-with-latent-weights~d9987409-132c-4232-8370-0f706fdd50bd
    """

    with h5py.File(weights_file, "r") as w:
        for l in model.layers:
            for _w in l.trainable_weights:
                print(tf.keras.backend.get_value(_w).flatten()[:3])
                tf.keras.backend.set_value(_w, 
                                           w["{}/{}".format(l.name, _w.name)])
                print(tf.keras.backend.get_value(_w).flatten()[:3])
                print("=====================")

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 

x_train = x_train.reshape(*x_train.shape, -1)
x_test = x_test.reshape(*x_test.shape, -1)

input_shape = x_train.shape[1:]
num_classes = 10

x_train = x_train / 255.0
y_train = np.eye(10)[y_train]

x_test = x_test / 255.0
y_test = np.eye(10)[y_test]

In [5]:
model1  = build_model(input_shape, num_classes)
model1.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# model1.summary()
train_log = model1.fit(x_train, y_train, epochs=2)
model1.save_weights("./model1.h5")
test_acc, test_loss = model1.evaluate(x_test, y_test, verbose=0)
print("Test Acc = ", test_acc)
print("Test Loss = ", test_loss)

Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 20s 341us/sample - loss: 0.3645 - accuracy: 0.8956
Epoch 2/2
60000/60000 [==============================] - 14s 236us/sample - loss: 0.3083 - accuracy: 0.9136
Test Acc =  0.29298651625514033
Test Loss =  0.9204


In [6]:
tf.keras.backend.clear_session()
tf.compat.v1.reset_default_graph()

In [7]:
model1 = build_model(input_shape, num_classes)
model1.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# model1.summary()
model1.load_weights("model1.h5")
test_acc, test_loss = model1.evaluate(x_test, y_test, verbose=0)
print("Test Acc = ", test_acc)
print("Test Loss = ", test_loss)

Test Acc =  0.29298651625514033
Test Loss =  0.9204


In [22]:
tf.keras.backend.clear_session()
tf.compat.v1.reset_default_graph()

In [23]:
model1 = build_model(input_shape, num_classes)
model1.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# model1.summary()
load_weights(model1, "./model1.h5")
test_acc, test_loss = model1.evaluate(x_test, y_test, verbose=0)
print("Test Acc = ", test_acc)
print("Test Loss = ", test_loss)

[-0.02724852 -0.1966636  -0.11848596]
[ 0.08475371 -0.10462408 -0.14177212]
[0. 0. 0.]
[-0.04077699 -0.00306944 -0.00033194]
[-0.08857512 -0.08443452  0.00768658]
[ 0.07939932 -0.02648997 -0.04596211]
[0. 0. 0.]
[-0.00559203 -0.00023849 -0.00438942]
[-0.01450286  0.01066554 -0.0019495 ]
[ 0.03025247 -0.02587075  0.03936682]
[0. 0. 0.]
[-0.07200084 -0.0292774   0.02811552]
Test Acc =  0.29298651625514033
Test Loss =  0.9204
